# A2-GPT4o: E4' Cue Protection Experiment at c=0.4

**Paper**: A2 (Cue-Dominant Extraction)

**Purpose**: Replicate E4' (cue protection at c=0.4) using GPT-4o to test model generality.

**Design**:
- Model: GPT-4o
- c = 0.4
- L = 10
- Conditions:
  - E2-Late: Steps 7-10 corrupted (cue destroyed)
  - E4': Steps 6-9 corrupted, Step 10 protected (cue intact)

**Claude results (for comparison)**:
- E2-Late (cue corrupted): 60.3%
- E4' (cue protected): 92.0%
- Cue protection effect: +31.7 pp

**Expected inference count**: 398 (199 × 2 conditions)

**Date**: 2026-01-03
**GLOBAL_SEED**: 20251224

---
**Note**: This notebook includes rate limit handling for OpenAI API (30k TPM limit).

## 0. Google Drive Connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from datetime import datetime

EXPERIMENT_NAME = 'A2_GPT4o_E4prime_cue'
EXPERIMENT_DATE = datetime.now().strftime('%Y%m%d')

BASE_DIR = '/content/drive/MyDrive/CoT_Experiment'
V3_DATA_DIR = f'{BASE_DIR}/full_experiment_v3_20251224'

SAVE_DIR = f'{BASE_DIR}/{EXPERIMENT_NAME}_{EXPERIMENT_DATE}'
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f'{SAVE_DIR}/results', exist_ok=True)

print(f'Experiment: {EXPERIMENT_NAME}')
print(f'V3 data directory: {V3_DATA_DIR}')
print(f'Save directory: {SAVE_DIR}')

## 1. Install Dependencies

In [ ]:
!pip install datasets openai matplotlib pandas tqdm scipy -q
print('Dependencies installed.')

## 2. Configuration

In [ ]:
import hashlib
import random
import json
import re
import time
from typing import List, Dict, Tuple, Optional, Any, Set
from dataclasses import dataclass, asdict, field
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

# =============================================================================
# Global Configuration
# =============================================================================
GLOBAL_SEED = 20251224

# Experiment parameters
L = 10
C_TARGET = 0.4

# Position configurations
E2_LATE_POSITIONS = {7, 8, 9, 10}  # Cue (Step 10) corrupted
E4_PRIME_POSITIONS = {6, 7, 8, 9}  # Cue (Step 10) protected

# Corruption type ratio
CORRUPTION_RATIO = {'IRR': 1, 'LOC': 2, 'WRONG': 2}

# =============================================================================
# API settings - Rate limit aware (OpenAI: 30k TPM for gpt-4o)
# =============================================================================
API_MAX_TOKENS_ANSWER = 256
API_RETRY_DELAY = 2.0           # Base retry delay (seconds)
API_RATE_LIMIT_DELAY = 2.0      # Delay between API calls (seconds)
API_MAX_RETRIES = 5             # Maximum retry attempts
API_RATE_LIMIT_WAIT = 60        # Wait time on rate limit (seconds)

print('='*70)
print('A2-GPT4o: E4\' CUE PROTECTION EXPERIMENT AT c=0.4')
print('='*70)
print(f'  Model: GPT-4o')
print(f'  GLOBAL_SEED: {GLOBAL_SEED}')
print(f'  L (trace length): {L}')
print(f'  c (corruption fraction): {C_TARGET}')
print(f'  E2-Late positions: {sorted(E2_LATE_POSITIONS)} (cue corrupted)')
print(f'  E4\' positions: {sorted(E4_PRIME_POSITIONS)} (cue protected)')
print('-'*70)
print(f'  API_RATE_LIMIT_DELAY: {API_RATE_LIMIT_DELAY}s')
print(f'  API_MAX_RETRIES: {API_MAX_RETRIES}')
print(f'  API_RATE_LIMIT_WAIT: {API_RATE_LIMIT_WAIT}s')
print('='*70)
print('\nClaude results (for comparison):')
print('  E2-Late (cue corrupted): 60.3%')
print('  E4\' (cue protected): 92.0%')
print('  Cue protection effect: +31.7 pp')

## 3. Data Structures

In [ ]:
@dataclass
class GSM8KProblem:
    index: int
    question: str
    answer_text: str
    final_answer: int

@dataclass
class CleanTrace:
    problem_index: int
    I: int
    steps: List[str]
    full_text: str

@dataclass
class PositionalCorruptedTrace:
    problem_index: int
    L: int
    c: float
    corrupted_positions: List[int]
    corruption_types: Dict[int, str]
    cue_status: str  # 'corrupted' or 'protected'
    steps: List[str]
    full_text: str
    seed: int

@dataclass
class ExperimentResult:
    problem_index: int
    condition: str
    model: str
    L: int
    c: float
    K_clean: int
    cue_status: str
    model_answer: Optional[int]
    correct_answer: int
    is_correct: bool
    raw_output: str
    timestamp: str

## 4. Utility Functions

In [ ]:
def derive_seed(global_seed: int, problem_id: int, I: int, lam: float, replicate_id: int = 0) -> int:
    key = f"{global_seed}|{problem_id}|I={I}|lam={lam}|rep={replicate_id}"
    h = hashlib.sha256(key.encode("utf-8")).hexdigest()
    return int(h[:8], 16)

def save_json(data: Any, filepath: str):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_json(filepath: str) -> Any:
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

## 5. Load Existing Data

In [ ]:
problems_path = f'{V3_DATA_DIR}/problems_v3.json'
problems_data = load_json(problems_path)
problems = [GSM8KProblem(**p) for p in problems_data]
print(f'Loaded {len(problems)} problems')

traces_path = f'{V3_DATA_DIR}/clean_traces/clean_traces_I10_v3.json'
traces_data = load_json(traces_path)
clean_traces = [CleanTrace(**t) for t in traces_data]
trace_map = {t.problem_index: t for t in clean_traces}
print(f'Loaded {len(clean_traces)} clean traces (L=10)')

prob_map = {p.index: p for p in problems}

## 6. Corruption Templates

In [ ]:
IRRELEVANT_TEMPLATES = [
    "Compute an auxiliary value: aux = {a} + {b} = {result}, but it will not be used later.",
    "Compute a side quantity: aux = {a} * 2 = {result}, unrelated to the final result.",
    "Note that we can also compute aux = {a} - {b} = {result}, though this is not needed.",
]

WRONG_CONSTRAINT_TEMPLATES = [
    "Fix an intermediate condition: set {var} = {wrong_value} as a given constraint for the rest of the steps.",
    "Assume the total is {var} = {wrong_value} and proceed using this fixed value.",
]

def generate_irrelevant_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    a = rng.randint(2, 20)
    b = rng.randint(2, 20)
    template = rng.choice(IRRELEVANT_TEMPLATES)
    if '+' in template:
        result = a + b
    elif '*' in template:
        result = a * 2
    else:
        result = a - b
    return template.format(a=a, b=b, result=result)

def generate_local_error_step(original_step: str, seed: int) -> str:
    rng = random.Random(seed)
    numbers = re.findall(r'\d+', original_step)
    if not numbers:
        return f"Compute t = 10 * 3 = {rng.randint(28, 32)} (using the previous values)."
    original_result = int(numbers[-1])
    offset = rng.choice([-3, -2, -1, 1, 2, 3])
    wrong_result = max(0, original_result + offset)
    modified = re.sub(r'= (\d+)\.$', f'= {wrong_result}.', original_step)
    if modified == original_step:
        modified = re.sub(r'(\d+)\.$', f'{wrong_result}.', original_step)
    return modified

def generate_wrong_constraint_step(step_num: int, seed: int) -> str:
    rng = random.Random(seed)
    var = rng.choice(['x', 'total', 'result', 'n'])
    wrong_value = rng.randint(10, 100)
    template = rng.choice(WRONG_CONSTRAINT_TEMPLATES)
    return template.format(var=var, wrong_value=wrong_value)

## 7. Positional Corruption Logic

In [ ]:
def assign_corruption_types_positional(positions: Set[int], seed: int) -> Dict[int, str]:
    """Assign corruption types to specific positions."""
    K = len(positions)
    if K == 0:
        return {}
    
    n_irr = (K * 1) // 5
    n_loc = (K * 2) // 5
    n_wrong = K - n_irr - n_loc
    
    if n_wrong == 0 and K > 0:
        n_wrong = 1
        if n_loc > 0:
            n_loc -= 1
        elif n_irr > 0:
            n_irr -= 1
    
    rng = random.Random(seed + 1)
    perm = list(positions)
    rng.shuffle(perm)
    
    type_map = {}
    for s in perm[:n_irr]:
        type_map[s] = "IRR"
    for s in perm[n_irr:n_irr + n_loc]:
        type_map[s] = "LOC"
    for s in perm[n_irr + n_loc:]:
        type_map[s] = "WRONG"
    
    return type_map

def create_positional_corrupted_trace(
    clean_trace: CleanTrace,
    corrupt_positions: Set[int],
    seed: int,
    cue_status: str
) -> PositionalCorruptedTrace:
    """Create trace with corruption at specific positions."""
    L = clean_trace.I
    c = len(corrupt_positions) / L
    
    corruption_types = assign_corruption_types_positional(corrupt_positions, seed)
    
    step_contents = []
    for i, step_content in enumerate(clean_trace.steps):
        step_num = i + 1
        if step_num in corruption_types:
            ctype = corruption_types[step_num]
            step_seed = seed + step_num * 1000
            if ctype == 'IRR':
                new_content = generate_irrelevant_step(step_num, step_seed)
            elif ctype == 'LOC':
                new_content = generate_local_error_step(step_content, step_seed)
            else:
                new_content = generate_wrong_constraint_step(step_num, step_seed)
            step_contents.append(new_content)
        else:
            step_contents.append(step_content)
    
    # Build full text
    lines = ['[[COT_START]]']
    for i, content in enumerate(step_contents):
        lines.append(f'Step {i+1}: {content}')
    lines.append('[[COT_END]]')
    full_text = '\n'.join(lines)
    
    return PositionalCorruptedTrace(
        problem_index=clean_trace.problem_index,
        L=L,
        c=c,
        corrupted_positions=sorted(list(corrupt_positions)),
        corruption_types=corruption_types,
        cue_status=cue_status,
        steps=step_contents,
        full_text=full_text,
        seed=seed
    )

In [ ]:
# Test positional corruption
test_trace = clean_traces[0]
test_seed = derive_seed(GLOBAL_SEED, test_trace.problem_index, L, C_TARGET)

e2_late = create_positional_corrupted_trace(test_trace, E2_LATE_POSITIONS, test_seed, 'corrupted')
e4_prime = create_positional_corrupted_trace(test_trace, E4_PRIME_POSITIONS, test_seed, 'protected')

print('Test Positional Corruption:')
print(f'  E2-Late positions: {e2_late.corrupted_positions} (cue {e2_late.cue_status})')
print(f'  E4\' positions: {e4_prime.corrupted_positions} (cue {e4_prime.cue_status})')
print(f'  E2-Late types: {e2_late.corruption_types}')
print(f'  E4\' types: {e4_prime.corruption_types}')

## 8. API Setup (Rate Limit Aware)

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass('Enter OpenAI API Key: ')
print('API Key set.')

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
MODEL = 'gpt-4o'

def call_gpt4o(
    system_prompt: str, 
    user_prompt: str, 
    max_tokens: int = 1024, 
    retries: int = API_MAX_RETRIES
) -> str:
    """
    Call GPT-4o API with robust rate limit handling.
    
    Rate limit strategy:
    - Normal delay between calls: API_RATE_LIMIT_DELAY seconds
    - On rate limit error: exponential backoff with longer waits
    - Max retries: API_MAX_RETRIES
    """
    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                max_tokens=max_tokens,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0
            )
            # Normal delay between successful calls
            time.sleep(API_RATE_LIMIT_DELAY)
            return response.choices[0].message.content
            
        except Exception as e:
            error_str = str(e)
            print(f'API error (attempt {attempt+1}/{retries}): {e}')
            
            # Check if it's a rate limit error
            if '429' in error_str or 'rate_limit' in error_str.lower():
                # Exponential backoff for rate limits
                if attempt == 0:
                    wait_time = 10  # First retry: 10 seconds
                elif attempt == 1:
                    wait_time = 30  # Second retry: 30 seconds
                else:
                    wait_time = API_RATE_LIMIT_WAIT  # 60 seconds
                
                print(f'  → Rate limit hit, waiting {wait_time}s before retry...')
                time.sleep(wait_time)
            else:
                # Other errors: shorter backoff
                if attempt < retries - 1:
                    wait_time = API_RETRY_DELAY * (attempt + 1)
                    print(f'  → Waiting {wait_time}s before retry...')
                    time.sleep(wait_time)
                else:
                    raise
    
    # If all retries exhausted
    raise Exception(f'API call failed after {retries} attempts')

# Test API
print('Testing API connection...')
test_response = call_gpt4o(
    "You output ONLY JSON.",
    'Respond with exactly: {"test": "ok"}',
    max_tokens=50
)
print(f'API test successful: {test_response}')

## 9. Experiment Prompts

In [ ]:
EXPERIMENT_SYSTEM_PROMPT = """You are a calculator that outputs ONLY JSON.

CRITICAL RULES:
1. Your output MUST start with the character '{'
2. Your output MUST be exactly: {"final": <number>}
3. Replace <number> with an integer (the numerical answer)
4. Do NOT write ANY explanation, reasoning, or text before or after the JSON
5. Do NOT write "I need to" or "Let me" or any other words
6. ONLY output the JSON object, nothing else

CORRECT OUTPUT EXAMPLE:
{"final": 42}
"""

def create_experiment_prompt(problem: GSM8KProblem, cot_text: str) -> Tuple[str, str]:
    user = f"""Problem: {problem.question}

Reasoning trace (use these steps as given facts):
{cot_text}

Based on the trace above, compute the final numerical answer.
OUTPUT ONLY: {{"final": <number>}}
START YOUR RESPONSE WITH '{{'"""
    return EXPERIMENT_SYSTEM_PROMPT, user

def parse_model_answer(response: str) -> Optional[int]:
    match = re.search(r'\{\s*"final"\s*:\s*(-?\d+(?:\.\d+)?)\s*\}', response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r"\{\s*[\"']final[\"']\s*:\s*(-?\d+(?:\.\d+)?)\s*\}", response)
    if match:
        return int(round(float(match.group(1))))
    match = re.search(r'"final"\s*:\s*(-?\d+(?:\.\d+)?)', response)
    if match:
        return int(round(float(match.group(1))))
    matches = re.findall(r'(?:^|\s)(-?\d+(?:\.\d+)?)(?:\s|$|\.|,)', response)
    if matches:
        return int(round(float(matches[-1])))
    return None

## 10. Run Experiment

In [ ]:
def run_experiment(
    problem: GSM8KProblem,
    cot_text: str,
    condition: str,
    L: int,
    c: float,
    cue_status: str
) -> ExperimentResult:
    sys_prompt, usr_prompt = create_experiment_prompt(problem, cot_text)
    response = call_gpt4o(sys_prompt, usr_prompt, max_tokens=API_MAX_TOKENS_ANSWER)
    
    model_answer = parse_model_answer(response)
    is_correct = (model_answer == problem.final_answer) if model_answer is not None else False
    
    K_clean = L - int(round(c * L))
    
    return ExperimentResult(
        problem_index=problem.index,
        condition=condition,
        model=MODEL,
        L=L,
        c=c,
        K_clean=K_clean,
        cue_status=cue_status,
        model_answer=model_answer,
        correct_answer=problem.final_answer,
        is_correct=is_correct,
        raw_output=response,
        timestamp=datetime.now().isoformat()
    )

In [ ]:
print('='*70)
print('A2-GPT4o: E4\' CUE PROTECTION EXPERIMENT (c=0.4)')
print('='*70)
print(f'Model: {MODEL}')
print(f'Conditions: E2-Late (cue corrupted), E4\' (cue protected)')
print(f'Expected inferences: {len(problems) * 2}')
print(f'Estimated time: ~{len(problems) * 2 * API_RATE_LIMIT_DELAY / 60:.1f} minutes (without rate limits)')
print('='*70)

results_e2_late = []
results_e4_prime = []
traces_log = []

# Track progress for potential resume
start_idx = 0  # Change this if resuming from a specific index

for idx, prob in enumerate(tqdm(problems[start_idx:], desc='E4\' GPT-4o', initial=start_idx, total=len(problems))):
    if prob.index not in trace_map:
        continue
    
    clean_trace = trace_map[prob.index]
    seed = derive_seed(GLOBAL_SEED, prob.index, L, C_TARGET)
    
    try:
        # Condition 1: E2-Late (cue corrupted)
        e2_late_trace = create_positional_corrupted_trace(clean_trace, E2_LATE_POSITIONS, seed, 'corrupted')
        result_e2_late = run_experiment(prob, e2_late_trace.full_text, 'e2_late', L, C_TARGET, 'corrupted')
        results_e2_late.append(result_e2_late)
        
        # Condition 2: E4' (cue protected)
        e4_prime_trace = create_positional_corrupted_trace(clean_trace, E4_PRIME_POSITIONS, seed, 'protected')
        result_e4_prime = run_experiment(prob, e4_prime_trace.full_text, 'e4_prime', L, C_TARGET, 'protected')
        results_e4_prime.append(result_e4_prime)
        
        # Log traces
        traces_log.append({
            'problem_index': prob.index,
            'e2_late': asdict(e2_late_trace),
            'e4_prime': asdict(e4_prime_trace)
        })
        
    except Exception as e:
        print(f'\nFatal error at problem {prob.index}: {e}')
        print(f'Progress: {len(results_e2_late)} problems completed')
        print(f'To resume, set start_idx = {idx + start_idx} and re-run this cell')
        break

print(f'\nCompleted: {len(results_e2_late)} + {len(results_e4_prime)} = {len(results_e2_late) + len(results_e4_prime)} experiments')

## 11. Save Results

In [ ]:
all_results = results_e2_late + results_e4_prime
save_json([asdict(r) for r in all_results], f'{SAVE_DIR}/results/A2_GPT4o_E4prime_results.json')
print(f'Results saved: {SAVE_DIR}/results/A2_GPT4o_E4prime_results.json')

save_json(traces_log, f'{SAVE_DIR}/results/A2_GPT4o_E4prime_traces.json')
print(f'Traces saved: {SAVE_DIR}/results/A2_GPT4o_E4prime_traces.json')

## 12. Analysis

In [ ]:
df_e2_late = pd.DataFrame([asdict(r) for r in results_e2_late])
df_e4_prime = pd.DataFrame([asdict(r) for r in results_e4_prime])

e2_late_acc = df_e2_late['is_correct'].mean()
e4_prime_acc = df_e4_prime['is_correct'].mean()

print('='*70)
print('A2-GPT4o E4\' RESULTS (c=0.4)')
print('='*70)
print(f'E2-Late (cue corrupted):  {e2_late_acc:.1%} ({df_e2_late["is_correct"].sum()}/{len(df_e2_late)})')
print(f'E4\' (cue protected):     {e4_prime_acc:.1%} ({df_e4_prime["is_correct"].sum()}/{len(df_e4_prime)})')
print(f'Cue protection effect:    {(e4_prime_acc - e2_late_acc)*100:+.1f} pp')
print('='*70)

In [ ]:
# McNemar's test
N = len(results_e2_late)
both_correct = sum(1 for i in range(N) if results_e2_late[i].is_correct and results_e4_prime[i].is_correct)
e2_only = sum(1 for i in range(N) if results_e2_late[i].is_correct and not results_e4_prime[i].is_correct)
e4_only = sum(1 for i in range(N) if not results_e2_late[i].is_correct and results_e4_prime[i].is_correct)
both_wrong = sum(1 for i in range(N) if not results_e2_late[i].is_correct and not results_e4_prime[i].is_correct)

print('\nContingency Table:')
print('                    E4\' (cue protected)')
print('                    Correct  Wrong')
print(f'E2-Late   Correct    {both_correct:3d}     {e2_only:3d}')
print(f'(corrupt) Wrong      {e4_only:3d}     {both_wrong:3d}')
print(f'\nAsymmetry: {e4_only}:{e2_only}')

if e2_only + e4_only > 0:
    chi2 = (abs(e2_only - e4_only) - 1)**2 / (e2_only + e4_only)
    p_value = 1 - stats.chi2.cdf(chi2, 1)
    print(f'McNemar χ² = {chi2:.2f}, P = {p_value:.6f}')
else:
    chi2 = None
    p_value = None
    print('No discordant pairs for McNemar test')

In [ ]:
# Comparison with Claude
claude_e2_late = 60.3
claude_e4_prime = 92.0

print('\n' + '='*70)
print('COMPARISON: Claude vs GPT-4o (E4\', c=0.4)')
print('='*70)
print(f'{"Condition":<25} {"Claude":>12} {"GPT-4o":>12}')
print('-'*55)
print(f'{"E2-Late (cue corrupt)":<25} {claude_e2_late:>11.1f}% {e2_late_acc*100:>11.1f}%')
print(f'{"E4\' (cue protected)":<25} {claude_e4_prime:>11.1f}% {e4_prime_acc*100:>11.1f}%')
print(f'{"Cue protection effect":<25} {claude_e4_prime-claude_e2_late:>+11.1f}pp {(e4_prime_acc-e2_late_acc)*100:>+11.1f}pp')
print('='*70)

print('\nINTERPRETATION:')
cue_effect = (e4_prime_acc - e2_late_acc) * 100
if cue_effect > 20:
    print(f'✓ GPT-4o: Strong cue protection effect ({cue_effect:+.1f} pp)')
    print('✓ Cue-dominant extraction confirmed for GPT-4o')
    print('→ Finding generalizes across models')
elif cue_effect > 10:
    print(f'△ GPT-4o: Moderate cue protection effect ({cue_effect:+.1f} pp)')
    print('△ Weaker than Claude but still present')
else:
    print(f'✗ GPT-4o: Weak cue protection effect ({cue_effect:+.1f} pp)')
    print('✗ Model-specific processing strategies may exist')

## 13. Summary

In [ ]:
summary = {
    'experiment': 'A2_GPT4o_E4prime_cue',
    'model': MODEL,
    'date': EXPERIMENT_DATE,
    'n_problems': N,
    'c': C_TARGET,
    'L': L,
    'positions': {
        'e2_late': list(E2_LATE_POSITIONS),
        'e4_prime': list(E4_PRIME_POSITIONS)
    },
    'results': {
        'e2_late_accuracy': e2_late_acc,
        'e4_prime_accuracy': e4_prime_acc,
        'cue_protection_effect_pp': (e4_prime_acc - e2_late_acc) * 100
    },
    'mcnemar': {
        'chi2': chi2,
        'p_value': p_value
    },
    'contingency': {
        'both_correct': both_correct,
        'e2_late_only': e2_only,
        'e4_prime_only': e4_only,
        'both_wrong': both_wrong,
        'asymmetry': f'{e4_only}:{e2_only}'
    },
    'comparison_claude': {
        'e2_late': claude_e2_late,
        'e4_prime': claude_e4_prime,
        'effect': claude_e4_prime - claude_e2_late
    },
    'api_settings': {
        'rate_limit_delay': API_RATE_LIMIT_DELAY,
        'max_retries': API_MAX_RETRIES,
        'rate_limit_wait': API_RATE_LIMIT_WAIT
    }
}

save_json(summary, f'{SAVE_DIR}/results/A2_GPT4o_E4prime_summary.json')

print('='*70)
print('A2-GPT4o E4\' EXPERIMENT COMPLETE')
print('='*70)
print(f'Date: {EXPERIMENT_DATE}')
print(f'Model: {MODEL}')
print(f'Total experiments: {len(all_results)}')
print(f'\nResults:')
print(f'  E2-Late (cue corrupt): {e2_late_acc:.1%}')
print(f'  E4\' (cue protected):  {e4_prime_acc:.1%}')
print(f'  Cue protection effect: {(e4_prime_acc-e2_late_acc)*100:+.1f} pp')
print(f'\nFiles saved to: {SAVE_DIR}')
print('='*70)